Model typu filtrowanie kolaboracyjne (collaborative filtering).

Wykorzystujemy w tym celu macierz interakcji, którą następnie dekomponujemy na macierze mniejszej wymiarowości z użyciem 
TruncatedSVD - sklearn. 

In [61]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

Przygotowanie macierzy interakcji

In [62]:
sessionsDataPath = '../notebooks/data/v2/sessions.jsonl'
productsDataPath = '../notebooks/data/v2/products.jsonl'
usersDataPath = '../notebooks/data/v2/users.jsonl'
sessionsDF = pd.read_json(sessionsDataPath, lines=True)
productsDF = pd.read_json(productsDataPath, lines=True)
usersDF = pd.read_json(usersDataPath, lines=True)

df = sessionsDF.drop(
    columns=["timestamp", "event_type", "offered_discount", "purchase_id"])
df["count"] = 1

#TODO Below split does not make any sense. Proper split need to be aplied for final build, but this one works as fast split for testing.
#BEGIN OF SHIT
train, test = train_test_split(df, test_size=0.2, stratify=df['user_id'])

#idea is to make sure that all data in test set has at least two entries from given session. Data that not satisfies that criteria are appended to train set.
s = test['session_id'].value_counts()
res = test[test['session_id'].map(s) >= 2]
ret = test[test['session_id'].map(s) == 1]
test.sort_values(by='session_id', inplace=True)
res.sort_values(by='session_id', inplace=True)
# display(test)
# display(res)
#END OF SHIT

train.append(ret).sort_values(by='session_id', inplace=True)
interactionMatrixDF = pd.pivot_table(train,
                                     index="user_id",
                                     columns="product_id",
                                     values="count",
                                     aggfunc=np.sum,
                                     fill_value=0)
display(interactionMatrixDF)


C:\Users\Bartosz\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


product_id,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,1310,1311,1312,1313,1314,1315,1316,1317,1318,1319
user_id,,,,,,,,,,,,,,,,,,,,,
102,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
103,0,2,4,14,1,3,2,3,2,9,...,0,0,2,0,1,3,5,1,0,6
104,0,1,5,3,5,9,0,4,3,6,...,1,2,1,0,0,1,5,2,6,1
105,0,1,5,8,4,5,0,4,3,5,...,0,1,1,0,1,0,8,2,0,1
106,3,1,1,5,2,3,2,2,1,6,...,0,0,2,2,1,1,6,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,3,0,4,12,6,2,0,3,8,5,...,0,1,0,2,0,1,8,1,2,1
298,0,0,8,1,2,3,0,4,1,2,...,1,0,0,0,1,1,7,2,0,0
299,0,0,2,0,1,0,0,0,0,1,...,0,0,1,0,2,0,3,1,0,1


Normalizacja wartości w macierzy interakcji.

In [63]:
#TODO do not know if scaler is required at all
interactionMatrixDF = pd.DataFrame(
    preprocessing.MinMaxScaler().fit_transform(interactionMatrixDF))
display(interactionMatrixDF)

,0,1,2,3,4,5,6,7,8,9,...,309,310,311,312,313,314,315,316,317,318
0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,...,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.142857,0.000000,0.000000
1,0.000000,0.4,0.266667,0.482759,0.090909,0.272727,0.5,0.272727,0.181818,0.5625,...,0.000000,0.000000,0.50,0.00,0.166667,0.75,0.178571,0.142857,0.000000,0.857143
2,0.000000,0.2,0.333333,0.103448,0.454545,0.818182,0.0,0.363636,0.272727,0.3750,...,0.333333,0.333333,0.25,0.00,0.000000,0.25,0.178571,0.285714,0.666667,0.142857
3,0.000000,0.2,0.333333,0.275862,0.363636,0.454545,0.0,0.363636,0.272727,0.3125,...,0.000000,0.166667,0.25,0.00,0.166667,0.00,0.285714,0.285714,0.000000,0.142857
4,0.428571,0.2,0.066667,0.172414,0.181818,0.272727,0.5,0.181818,0.090909,0.3750,...,0.000000,0.000000,0.50,0.50,0.166667,0.25,0.214286,0.428571,0.000000,0.142857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.428571,0.0,0.266667,0.413793,0.545455,0.181818,0.0,0.272727,0.727273,0.3125,...,0.000000,0.166667,0.00,0.50,0.000000,0.25,0.285714,0.142857,0.222222,0.142857
196,0.000000,0.0,0.533333,0.034483,0.181818,0.272727,0.0,0.363636,0.090909,0.1250,...,0.333333,0.000000,0.00,0.00,0.166667,0.25,0.250000,0.285714,0.000000,0.000000
197,0.000000,0.0,0.133333,0.000000,0.090909,0.000000,0.0,0.000000,0.000000,0.0625,...,0.000000,0.000000,0.25,0.00,0.333333,0.00,0.107143,0.142857,0.000000,0.142857
198,0.428571,0.6,0.533333,0.241379,0.181818,0.363636,0.0,0.363636,0.363636,0.3750,...,0.333333,0.166667,0.00,0.00,0.333333,1.00,0.214286,0.142857,0.000000,0.428571


Utworzenie słownika dla szybszego wyszukiwania.

In [64]:
idxNameDict = pd.Series(productsDF["product_id"].values,
                        index=productsDF.index).to_dict()
idxUserDict = pd.Series(usersDF["user_id"].values,
                        index=usersDF.index).to_dict()


Dekompozycja utworzonej macierzy na podmacierze ze względu na użytkowników i produkty.

In [65]:
from sklearn.decomposition import TruncatedSVD

#initial hiperparameters
epsilon = 1e-9
latentFactors = 10

#generate user latent features
userSVD = TruncatedSVD(n_components=latentFactors)
userFeatures = userSVD.fit_transform(interactionMatrixDF) + epsilon

#generate item latent features
itemSVD = TruncatedSVD(n_components=latentFactors)
itemFeatures = itemSVD.fit_transform(interactionMatrixDF.transpose(
)) + epsilon  #transpose because items are columns

pd.DataFrame(itemFeatures)
pd.DataFrame(userFeatures)


,0,1,2,3,4,5,6,7,8,9
0,0.357325,-0.016516,-0.036711,-0.013436,0.019591,0.031847,0.035075,0.035796,0.027358,-0.046832
1,4.397823,0.951287,0.860339,0.193843,-0.003095,1.672106,-1.179329,0.171740,-0.622493,-0.786968
2,3.714773,-0.315986,-0.531188,0.202495,0.294843,-0.671277,0.604875,0.574376,0.005633,-0.409986
3,5.543929,0.916961,-0.994871,-0.265284,-1.107186,0.554274,0.960323,0.272213,-0.596037,0.401094
4,3.619734,-0.567563,0.228599,-0.773170,-0.025958,0.741643,0.264943,0.145590,0.562818,0.545819
...,...,...,...,...,...,...,...,...,...,...
195,4.525057,0.349488,-0.155552,0.492791,0.020202,0.136724,0.435584,-1.144263,-0.345886,0.161606
196,2.589297,-0.229572,-0.802327,0.168718,-0.083028,-0.039368,0.044450,-0.021443,-0.242001,-0.183567
197,1.001325,-0.024071,0.168213,-0.235846,-0.008467,0.113974,0.026633,0.012734,0.082252,-0.049721
198,3.556453,-0.791697,-0.396768,0.148154,-0.078462,0.341335,-0.442306,-0.628925,0.189681,-0.432914


In [66]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=8, n_init=1000)

model = kmeans.fit_predict(userFeatures)
modelDF = pd.DataFrame(model).rename(columns={0: 'group'}, index=idxUserDict)
modelDF


,group
102,0
103,3
104,3
105,6
106,1
...,...
297,5
298,2
299,0
300,1


Budowanie profili użytkowników  

In [67]:
separator = ';'
newGroups = [
    'Gry komputerowe', 'Gry na konsole', 'Sprzęt RTV', 'Komputery',
    'Telefony i akcesoria'
]


def castCategoryPath(categoryPath):
    categories = categoryPath.split(separator)
    foundGroups = [group for group in newGroups if group in categories]
    if len(foundGroups) != 1:
        raise RuntimeError('wrong group cast: {}'.format(foundGroups))
    return foundGroups[0]

In [142]:
def build_user_profile(userID: int, merged: pd.DataFrame) -> dict:
    profile = {}
    userActDF = merged.loc[merged['user_id'] == userID].drop(
        columns=['user_id'])

    for group in newGroups:
        g = userActDF.loc[userActDF['category_path'] == group].drop(
            columns=['category_path']).value_counts().to_frame().rename(
                columns={
                    0: 'count'
                }).reset_index()
        g = g[g['count'] >= g['count'].quantile(0.9)]
        profile[group] = g.set_index('product_id').to_dict()
    return profile


In [156]:
merged = pd.merge(sessionsDF, productsDF, how='inner', on='product_id')
merged.drop(columns=[
    'session_id', "timestamp", "event_type", "offered_discount", "purchase_id",
    'product_name', 'price', 'user_rating'
],
            inplace=True)
merged['category_path'] = merged['category_path'].apply(castCategoryPath)

usersID = usersDF['user_id'].unique()
profileDict = {}

for userID in usersID:
    profileDict[userID] = build_user_profile(userID, merged)

#konieczne jest przepakowanie dict'a, gdyż konwersja z int64 na int ma w numpy'u buga, który dokonuje konwersji int64 na int32. Problem otwarty na Github'ie, tylko Windows.
profileDict = {int(key): value for key, value in profileDict.items()}


Zapis modelu i profili do plików.

In [147]:
import json

modelDF.to_json('../models/similar_users.json')

with open('../models/profiles.json', 'w') as file:
    json.dump(profileDict, file, sort_keys=True, indent=4)


Testowe odczytanie danych

In [148]:
savedModel = pd.read_json('../models/similar_users.json')

with open('../models/profiles.json', 'r') as file:
    data = json.load(file)

savedProfiles = {int(key): value for key, value in data.items()}

display(savedModel)
savedProfiles[102]

,group
102,0
103,3
104,3
105,6
106,1
...,...
297,5
298,2
299,0
300,1


{'Gry komputerowe': {'count': {'1049': 6, '1050': 14, '1053': 6, '1054': 6}},
 'Gry na konsole': {'count': {'1041': 2}},
 'Komputery': {'count': {'1017': 3, '1077': 3, '1276': 3}},
 'Sprzęt RTV': {'count': {'1233': 3}},
 'Telefony i akcesoria': {'count': {'1317': 1}}}

Przykład predykcji  
Użytkownik o id 102 przegląda produkt z kategorii *Gry komputerowe*

Poniżej predykcja na modelu w pamięci.

In [155]:
input = [102, 'Gry komputerowe']

group = modelDF.loc[modelDF['group'] == modelDF.iloc[102]['group']].index
dfListPre = []

for user in group:
    dfListPre.append(pd.DataFrame(profileDict[user][input[1]]))

dfList = [
    df.reset_index().rename(columns={'index': 'product_id'})
    for df in dfListPre
]

df = pd.concat(dfList).groupby(['product_id']).sum().reset_index().sort_values(
    by=['count'], ascending=False)
df

,product_id,count
5,1053,1767
2,1050,1496
0,1048,1046
1,1049,686
7,1055,632
4,1052,577
6,1054,464
8,1056,456
3,1051,214
10,1272,159


Predykcja dla modelu odczytanego z .json.  
Predykcje są identyczne, stąd wnioskujemy poprawnośc serializacji.

In [153]:
input = [102, 'Gry komputerowe']

group = savedModel.loc[savedModel['group'] == savedModel.iloc[102]
                       ['group']].index
dfListPre = []

for user in group:
    dfListPre.append(pd.DataFrame(savedProfiles[user][input[1]]))

dfList = [
    df.reset_index().rename(columns={'index': 'product_id'})
    for df in dfListPre
]

df = pd.concat(dfList).groupby(['product_id']).sum().reset_index().sort_values(
    by=['count'], ascending=False)
df

,product_id,count
5,1053,1767
2,1050,1496
0,1048,1046
1,1049,686
7,1055,632
4,1052,577
6,1054,464
8,1056,456
3,1051,214
10,1272,159
